In [310]:
import random
from numpy import linspace
import itertools

In [268]:
%run Extraccion.ipynb

In [269]:
%run ParserMaterias.ipynb

In [270]:
get_recommendations("topologia ecuaciones diferenciales")

[162, 765, 183, 165, 766, 395, 433, 163, 712, 939]

In [271]:
def get_recomendations_str(intereses):
    '''
    Dado una cadena con los intereses de alguna persona, regresa el nombre
    de las materias que le recomienda.
    ----------------------------------------------------------
    :param intereses str: Cadena que tiene intereses del alumno
    :returns [str]: Lista con materias a considerar
    '''
    recomendaciones = []
    recomendaciones_num = get_recommendations(intereses)
    for recomendacion in recomendaciones_num:
        try:
            recomendaciones.append((clave_to_nombre(recomendacion),str_to_clave(recomendacion)))
        except Exception as e:
            pass
    return recomendaciones

In [272]:
get_recomendations_str('topologia ecuaciones diferenciales')

[('ecuaciones diferenciales i', '0162'),
 ('topologia i', '0765'),
 ('ecuaciones diferenciales parciales ii', '0183'),
 ('ecuaciones diferenciales parciales i', '0165'),
 ('topologia ii', '0766'),
 ('ecuaciones diferenciales ii', '0163'),
 ('seminario de topologia b', '0712')]

In [273]:
def generar_candidatos(claves,hora_inicio,hora_fin,dia):
    '''
    Para las claves e información sobre hora y día, regresa un diccionario con
    los grupos disponibles para esas materias en esas restricciones, es decir
    un diccionario cuyas llaves son intervalos de horas y sus valores los grupos
    que tienen clase a esa hora en ese día, eligiendo intervalos de horas entre
    la hora de inicio y fin establecidas.
    -------------------------------------------------
    :param claves [str]: Lista con las claves de materias a recuperar
    :hora_inicio float: Hora a la que inicia el horario debe ser entero o
                        un entero y medio (e.g 9.0 y 9.50 son válidos, mientras que
                        6:35 no es una hora válida) el 0.5 representa la media
                        hora.
    :hora_fin float: Hora máxima a la que termina el horario

    '''
    if hora_inicio%0.5 != 0 or hora_fin%0.5 != 0:
        raise Exception('Ese no es un horario válido, tiene que representar una hora o \n fracción de media hora')
        
    
        
    horario_dia = {}
    horas_disponibles = set()
    grupos_por_clave = {}
    #Un intervalo de las horas representadas en fracciones de media hora
    horas_eleccion =  linspace(hora_inicio,hora_fin,int(2*(hora_fin-hora_inicio)+1))
    
    #for dia in dias.keys():
    for clave in claves:
        grupos_por_clave[clave] = []
        for hora in horas_eleccion:
            horario = get_horario(clave,hora,dia)

            if horario is False: #si no hay grupos disponibles a esa hora
                continue
            if horario[0][1]>hora_fin: #Si el horario se pasa de la hora establecida
                continue

            hora_de_clase = horario[0]
            grupos = horario[1]
            if hora_de_clase not in horario_dia.keys():
                horario_dia[hora_de_clase] = []
                
            horas_disponibles.add(hora_de_clase)
            horario_dia[hora_de_clase] += grupos
            grupos_por_clave[clave] += grupos
    
    return horario_dia,list(horas_disponibles), grupos_por_clave

candidatos = generar_candidatos(['0002','0003','0083','0163','0715'],8.0,12,'lu')
candidatos

({(10.0, 11.0): ['4212', '4213'], (9.0, 10.0): ['4214', '4214']},
 [(9.0, 10.0), (10.0, 11.0)],
 {'0002': ['4212', '4213'],
  '0003': ['4214', '4214'],
  '0083': [],
  '0163': [],
  '0715': []})

In [316]:
def generar_candidatos_semana(claves, hora_inicio, hora_fin):
    dias = ['lu','ma','mi','ju','vi','sa']
    
    horario_por_dia = {}
    horas_disponibles_por_dia = {}
    grupos_por_clave = {}
    for dia in dias:
        candidatos_dia = generar_candidatos(claves,hora_inicio,hora_fin,dia)
        horario_por_dia[dia] = candidatos_dia[0]
        horas_disponibles_por_dia[dia] = candidatos_dia[1]
        for intervalo in candidatos_dia[2].keys():
            if intervalo not in grupos_por_clave.keys():
                grupos_por_clave[intervalo] = []
            grupos_por_clave[intervalo]+= candidatos_dia[2][intervalo]
    
    grupos_por_clave = {intervalo:list(set(grupos_por_clave[intervalo]))
                       for intervalo in grupos_por_clave.keys()}
    return horario_por_dia, horas_disponibles_por_dia, grupos_por_clave

candidatos = generar_candidatos_semana(['0002','0003','0005','0715'],6.0,10)

In [332]:
def se_intersecta(intervalo,horario_dia):
    '''
    Dado un intervalo de tiempo y un horario, checa si alguna
    clase del horario se intersecta con el intervalo.
    '''
    horario_lista = sorted(list(horario_dia.keys()),key= lambda x:x[1])
    inicio = intervalo[0]
    fin = intervalo[1]
    for hora in horario_lista:
        hora_inicio = hora[0]
        hora_fin = hora[1]
        if hora_inicio <= inicio < hora_fin or hora_inicio < fin <= hora_fin:
            return True
    return False

def se_intersecta_semana(intervalos,horario):
    '''
    Dado un horario de materia por día y un horario, checa si alguna
    clase del horario se intersecta con la materia.
    '''
    dias = ['lu','ma','mi','ju','vi','sa']
    for dia in horario.keys():
        if dia in intervalos.keys():
            horario_dia = horario[dia]
            if se_intersecta(intervalos[dia],horario_dia):
                return True
    return False

prueba_dias = candidatos[0]
horario_prueba = grupo_info('4174')[2]
se_intersecta_semana(horario_prueba,prueba_dias)

print(prueba_dias)
print(horario_prueba.items())

{'lu': {(7.0, 8.0): ['4211', '4110', '4111'], (9.0, 10.0): ['4214', '4214'], (8.0, 9.0): ['4112', '4113']}, 'ma': {(7.0, 8.0): ['4211', '4110', '4111'], (9.0, 10.0): ['4214', '4214'], (8.0, 9.0): ['4112', '4113'], (7.0, 10.0): ['8242', '8244', '8242', '8244']}, 'mi': {(7.0, 8.0): ['4211', '4110', '4111'], (9.0, 10.0): ['4214', '4214'], (8.0, 9.0): ['4112', '4113'], (7.0, 10.0): ['8246', '8246']}, 'ju': {(7.0, 8.0): ['4211', '4110', '4111'], (9.0, 10.0): ['4214', '4214'], (8.0, 9.0): ['4112', '4113'], (7.0, 10.0): ['8242', '8244', '8242', '8244']}, 'vi': {(7.0, 8.0): ['4211', '4110', '4111'], (9.0, 10.0): ['4214', '4214'], (8.0, 9.0): ['4112', '4113'], (7.0, 10.0): ['8246', '8246']}, 'sa': {}}
dict_items([('lu', (7.0, 8.0)), ('ma', (7.0, 8.0)), ('mi', (7.0, 8.0)), ('ju', (7.0, 8.0)), ('vi', (7.0, 8.0))])


In [276]:
lista = ['a','b']
lista.remove('a')
lista

['b']

In [300]:
lista = ['a','b']
random.shuffle(lista)
lista

['a', 'b']

In [360]:


def conjunto_se_intersecta(grupos):
    '''
    Dado un conjunto de grupos, comprueba si los horarios
    de los grupos se intersectan
    -------------------------------------------------
    :param grupos [str]: Lista de grupos
    
    :return Bool: Indicador de si los conjuntos se intersectan o no.
    '''
    dias = ['lu','ma','mi','ju','vi','sa']
    primer_horario = grupo_info(grupos[0])[2]
    horario = {dia:{} for dia in dias}
    
    for dia, hora in primer_horario.items():
        horario[dia][hora] = grupos[0]
    
    for grupo in grupos[1:]:
        horario_materia = grupo_info(grupo)[2]
        if se_intersecta_semana(horario_materia,horario):
            return True, horario
        for dia, hora in horario_materia.items():
            horario[dia][hora] = grupo

    return False, horario
conjunto_se_intersecta(['4171','4172'])

(False,
 {'lu': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'ma': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'mi': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'ju': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'vi': {(15.0, 16.0): '4171', (19.0, 20.0): '4172'},
  'sa': {}})

In [376]:
def crear_horario(claves,hora_inicio,hora_fin,seed=None):
    '''
    Dada una lista de claves y hora de inicio y fin de un horario deseado,
    regresa un diccionario con el horario de esas materias, así como
    una lista de (clave,grupo) en caso de que sea posible formar el horario,
    en caso contrario, regresa Falso
    ----------------------------------------------------------------
    :param claves [str]: Lista de claves
    :param hora_inicio float: Hora de inicio de las clases representadas
                             en horas u horas y media (XX.0, XX.5)
    :param hora_fin float: Hora de fin de las clases representadas
                             en horas u horas y media (XX.0, XX.5)
    :param seed int: Semilla para la elección de grupos, en caso de no ser especificada,
                    la elección de grupos será aleatoria
    
    :returns False: En caso de no haber una combinación posible
    :returns horario dict: Diccionario con los días y las horas en que se tiene cada
                        materia. Ejem:
                        {'lu': {(9.0, 10.0): '4175', (8.0, 9.0): '4112'},
                          'ma': {(9.0, 10.0): '4175', (8.0, 9.0): '4112'},
                          'mi': {(9.0, 10.0): '4175', (8.0, 9.0): '4112'},
                          'ju': {(9.0, 10.0): '4175', (8.0, 9.0): '4112'},
                          'vi': {(9.0, 10.0): '4175', (8.0, 9.0): '4112'},
                          'sa': {}}
    :returns [(clave,grupo)]: Lista con las claves y grupos que se eligieron. 
                            Ejem: [('0001', '4175'), ('0005', '4112')]
    '''
    if hora_inicio%0.5 != 0:
        raise Exception('Ese no es un horario válido, tiene que representar una hora o \n fracción de media hora')
        
    if seed is None:
        seed = random.randint(0,2**10)
    random.seed(seed)
    random.shuffle(claves)
    
    candidatos = generar_candidatos_semana(claves,hora_inicio,hora_fin)
    candidatos_por_clave = candidatos[2]
    for dia in candidatos_por_clave.keys():
        if candidatos_por_clave[dia] == []:
            return False #En caso de que no sea posible acomodar algún horario
        random.shuffle(candidatos_por_clave[dia])
    
    grupos = list(candidatos_por_clave.values())
    combinaciones = list(itertools.product(*grupos))
    for comb in combinaciones:
        if not conjunto_se_intersecta(comb)[0]:
            return conjunto_se_intersecta(comb)[1], list(zip(claves,comb))
    return False

crear_horario(['0766','0001'],8,13)

({'lu': {(9.0, 10.0): '4176', (10.0, 11.0): '4316'},
  'ma': {(9.0, 10.0): '4176', (10.0, 11.0): '4316'},
  'mi': {(9.0, 10.0): '4176', (10.0, 11.0): '4316'},
  'ju': {(9.0, 10.0): '4176', (10.0, 11.0): '4316'},
  'vi': {(9.0, 10.0): '4176', (10.0, 11.0): '4316'},
  'sa': {}},
 [('0001', '4176'), ('0766', '4316')])